In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
import time
from nltk.corpus import wordnet
import random
from optuna import create_study, trial
from sklearn.model_selection import KFold

file_name = 'data_2/combined_file.csv' # 감정 세기 데이터셋

# CSV 파일 로드
print("CSV 파일 로드 중...")
df = pd.read_csv(file_name)
print(f"데이터 로드 완료. 샘플 수: {len(df)}")

# '발화문'과 '1번 감정'만 사용
X = df['text']
y = df['emotion']

# 레이블 인코딩
print("레이블 인코딩 중...")
le = LabelEncoder()
y = le.fit_transform(y)
print(f"고유 감정 레이블 수: {len(le.classes_)}")

# 마침표와 쉼표를 제거하는 함수 정의
def remove_punctuation(text):
    return text.replace('.', '').replace(',', '')

# 'text' 열에 함수 적용
X = X.apply(remove_punctuation)

print(X)

# 불용어 목록 정의
stopwords = set([
    '는', '은', '이', '가', '을', '를', '에', '의', '도', '에서', '와', '과', '로', '으로',
    '그리고', '그러나', '그런데', '하지만', '또는', '그래서', '따라서',
    '이것', '저것', '그것', '우리', '여러분', '나', '너', '그', '그녀',
    '입니다', '있습니다', '합니다', '되었습니다', '있다', '것이다', '합니다'
])

# 불용어 제거 함수 정의
def remove_stopwords(text):
    if isinstance(text, str):
        words = text.split()
        meaningful_words = [word for word in words if word not in stopwords]
        return ' '.join(meaningful_words)
    return text

# 'text' 열에 함수 적용
X = X.apply(remove_punctuation)

print(X)


# 데이터 분할
print("데이터 분할 중...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"학습 데이터 수: {len(X_train)}, 테스트 데이터 수: {len(X_test)}")

# KLUE/BERT 모델 및 토크나이저 로드
print("KLUE/BERT 모델 및 토크나이저 로드 중...")
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# GPU 사용 가능 시 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)
print(f"모델 로드 완료. 사용 중인 디바이스: {device}")


# 데이터 토큰화 및 모델 입력 준비 함수
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# PyTorch 텐서를 numpy 배열로 변환하는 함수
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()

# BERT 임베딩을 생성하는 함수 (배치 처리)
def get_embeddings(texts, batch_size=32):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="임베딩 생성 중"):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(embeddings)
    return np.vstack(all_embeddings)

# 학습 및 테스트 데이터에 대한 BERT 임베딩 생성
print("학습 데이터 임베딩 생성 중...")
start_time = time.time()
X_train_emb = get_embeddings(X_train.tolist())
print(f"학습 데이터 임베딩 생성 완료. 소요 시간: {time.time() - start_time:.2f}초")

print("테스트 데이터 임베딩 생성 중...")
start_time = time.time()
X_test_emb = get_embeddings(X_test.tolist())
print(f"테스트 데이터 임베딩 생성 완료. 소요 시간: {time.time() - start_time:.2f}초")


CSV 파일 로드 중...
데이터 로드 완료. 샘플 수: 79813
레이블 인코딩 중...
고유 감정 레이블 수: 6
0                                         어 청소 니가 대신 해 줘!
1                                        둘 다 청소 하기 싫어 귀찮아
2                                           둘 다 하기 싫어서 화내
3                                              그럼 방세는 어떡해
4                                권태긴줄 알았는데 다른 사람이 생겼나보더라고
                               ...                       
79808           이제 몸이 점점 약해진다는 게 느껴져 아내에게 미안하고 속상한 마음이 들어
79809    나이가 다 돼서 퇴직을 하게 되었어 하지만 노후 준비를 제대로 안 해서 돈 걱정이 앞서
79810        나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해 능력도 없고
79811           몸이 많이 약해졌나 봐 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나
79812     몇십 년을 함께 살았던 남편과 이혼했어 그동안의 세월에 배신감을 느끼고 너무 화가 나
Name: text, Length: 79813, dtype: object
0                                         어 청소 니가 대신 해 줘!
1                                        둘 다 청소 하기 싫어 귀찮아
2                                           둘 다 하기 싫어서 화내
3                                              그럼 방세는 어떡해
4                      

임베딩 생성 중: 100%|██████████| 1996/1996 [01:33<00:00, 21.44it/s]


학습 데이터 임베딩 생성 완료. 소요 시간: 93.17초
테스트 데이터 임베딩 생성 중...


임베딩 생성 중: 100%|██████████| 499/499 [00:22<00:00, 22.01it/s]

테스트 데이터 임베딩 생성 완료. 소요 시간: 22.69초


In [2]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean

# 예시 데이터 (여기서는 X_train_emb와 y_train이 이미 정의되어 있다고 가정)
# 3차원 t-SNE로 차원 축소
tsne_3d = TSNE(n_components=3, random_state=42)
X_tsne_3d = tsne_3d.fit_transform(X_train_emb)

# 데이터프레임 생성
df = pd.DataFrame({
    'tsne-3d-1': X_tsne_3d[:, 0],
    'tsne-3d-2': X_tsne_3d[:, 1],
    'tsne-3d-3': X_tsne_3d[:, 2],
    'emotion': [le.classes_[i] for i in y_train],
    'text': X_train.tolist()
})

# 결과를 저장할 딕셔너리
outliers_dict = {}

# 감정별 클러스터링 및 이상치 탐지
for emotion in df['emotion'].unique():
    emotion_df = df[df['emotion'] == emotion]

    # K-평균 클러스터링을 사용하여 클러스터 중심 계산
    kmeans = KMeans(n_clusters=1, random_state=42)
    kmeans.fit(emotion_df[['tsne-3d-1', 'tsne-3d-2', 'tsne-3d-3']])
    cluster_center = kmeans.cluster_centers_[0]

    # 각 포인트와 클러스터 중심 사이의 거리 계산
    distances = emotion_df.apply(
        lambda row: euclidean((row['tsne-3d-1'], row['tsne-3d-2'], row['tsne-3d-3']), cluster_center),
        axis=1
    )

    # 거리의 평균과 표준편차 계산
    mean_distance = distances.mean()
    std_distance = distances.std()

    # 이상치: 평균 거리에서 2 표준편차 이상 떨어진 포인트들
    outliers = emotion_df[distances > mean_distance + 2 * std_distance]

    # 이상치의 거리 비율 계산
    outlier_distances = distances[distances > mean_distance + 2 * std_distance]
    distance_ratios = outlier_distances / mean_distance

    # 이상치 인덱스 추적
    outlier_indices = outliers.index.tolist()

    # 이상치 데이터를 좌표를 키로 하는 딕셔너리에 저장
    for idx, (outlier, distance_ratio) in enumerate(zip(outliers.iterrows(), distance_ratios)):
        index, row = outlier
        # 좌표를 키로 사용
        key = (row['tsne-3d-1'], row['tsne-3d-2'], row['tsne-3d-3'])
        outliers_dict[key] = {
            'text': row['text'],
            'emotion': row['emotion'],
            'distance_ratio': distance_ratio,
            'index': index
        }

# 딕셔너리 출력 (디버깅 목적)
for key, value in outliers_dict.items():
    print(f"Coordinates: {key}")
    print(f"  Text: {value['text']}")
    print(f"  Emotion: {value['emotion']}")
    print(f"  Distance Ratio: {value['distance_ratio']:.2f}")
    print(f"  Index: {value['index']}\n")

# 딕셔너리를 데이터프레임으로 변환하여 CSV 파일로 저장
outliers_list = [
    {
        'Coordinates': key,
        'Text': value['text'],
        'Emotion': value['emotion'],
        'Distance Ratio': value['distance_ratio'],
        'Index': value['index']
    }
    for key, value in outliers_dict.items()
]

outliers_df = pd.DataFrame(outliers_list)

# CSV 파일로 저장
outliers_df.to_csv('outliers_data.csv', index=False, encoding='utf-8-sig')

print("Data has been successfully saved to 'outliers_data.csv'")


Coordinates: (-42.18119812011719, 29.48708152770996, -4.5552167892456055)
  Text: 노후 준비가 도저히 힘들어서 포기할까 봐
  Emotion: 슬픔
  Distance Ratio: 1.66
  Index: 5998

Coordinates: (28.015888214111328, -41.74417495727539, 4.852357387542725)
  Text: 강남에서 보기로 했는데 요즘 바이러스 되게 심각하잖아 불안해져가지고
  Emotion: 슬픔
  Distance Ratio: 1.65
  Index: 9562

Coordinates: (4.580154895782471, -1.3263919353485107, 50.92375946044922)
  Text: 이젠 더이상 가족들에게 피해를 줄 수 없어 어떻게든 퇴원하기로 했어
  Emotion: 슬픔
  Distance Ratio: 1.67
  Index: 11735

Coordinates: (44.28794860839844, 1.337058424949646, -15.689961433410645)
  Text: 내가 열심히 작업했는데 다 잘못 됐다고 하니까 너무 속상했어 하지만 오늘 열심히 해서 내 실력을 인정받는 날이 왔으면 좋겠어!
  Emotion: 슬픔
  Distance Ratio: 1.65
  Index: 13568

Coordinates: (44.26064682006836, 18.648897171020508, 1.5830488204956055)
  Text: 나 이벤트 당첨됐어	
  Emotion: 슬픔
  Distance Ratio: 1.70
  Index: 16586

Coordinates: (45.31499481201172, -9.396368026733398, -11.370447158813477)
  Text: 고마워 근데 쉴 시간이 없어 당장 다시 해오래
  Emotion: 슬픔
  Distance Ratio: 1.65
  In

In [4]:
# CSV 파일로 저장
outliers_df.to_csv('outliers_data2.csv', index=False)

print("Data has been successfully saved to 'outliers_data.csv'")


Data has been successfully saved to 'outliers_data.csv'


In [10]:
df = pd.read_csv('outliers_data2.csv')
df

,tsne-3d-1,tsne-3d-2,tsne-3d-3,text,emotion,distance_ratio
0,-42.181198,29.487082,-4.555217,노후 준비가 도저히 힘들어서 포기할까 봐,슬픔,1.657373
1,28.015888,-41.744175,4.852357,강남에서 보기로 했는데 요즘 바이러스 되게 심각하잖아 불안해져가지고,슬픔,1.651410
2,4.580155,-1.326392,50.923759,이젠 더이상 가족들에게 피해를 줄 수 없어 어떻게든 퇴원하기로 했어,슬픔,1.671543
3,44.287949,1.337058,-15.689961,내가 열심히 작업했는데 다 잘못 됐다고 하니까 너무 속상했어 하지만 오늘 열심히 해...,슬픔,1.649911
4,44.260647,18.648897,1.583049,나 이벤트 당첨됐어\t,슬픔,1.703185
...,...,...,...,...,...,...
1153,-4.181704,23.597727,-37.426121,친구들이 쉬는 시간마다 매점 심부름을 시켜 안 그랬으면 좋겠어,분노,1.741755
1154,0.712044,40.672192,1.101676,다행이 내가 큰 병이 아니라서 병원비는 매달 나오는 연금으로 충분히 메꿀 수 있을 ...,분노,1.697193
1155,10.455350,38.852051,2.598256,살이 쪄서 건강 검진받을 때마다 조마조마했는데 큰 병 없이 진단 결과가 나와서 참 ...,분노,1.679213
1156,-5.172163,22.661451,-35.919083,애인이 잘못한 것을 다른 사람들이 다 알았으면 좋겠어,분노,1.686175


In [13]:
df['text'] = df['text'].str.replace('\t', '', regex=True)
df.to_csv('outliers_data2.csv', index=False)


In [15]:
df.head(50)

,tsne-3d-1,tsne-3d-2,tsne-3d-3,text,emotion,distance_ratio
0,-42.181198,29.487082,-4.555217,노후 준비가 도저히 힘들어서 포기할까 봐,슬픔,1.657373
1,28.015888,-41.744175,4.852357,강남에서 보기로 했는데 요즘 바이러스 되게 심각하잖아 불안해져가지고,슬픔,1.651410
2,4.580155,-1.326392,50.923759,이젠 더이상 가족들에게 피해를 줄 수 없어 어떻게든 퇴원하기로 했어,슬픔,1.671543
3,44.287949,1.337058,-15.689961,내가 열심히 작업했는데 다 잘못 됐다고 하니까 너무 속상했어 하지만 오늘 열심히 해...,슬픔,1.649911
4,44.260647,18.648897,1.583049,나 이벤트 당첨됐어,슬픔,1.703185
5,45.314995,-9.396368,-11.370447,고마워 근데 쉴 시간이 없어 당장 다시 해오래,슬픔,1.653671
6,33.618263,-24.114624,-28.153883,우리는 지금 빠르게 변하는 시대에 살고 있지 않은가?,슬픔,1.686612
7,26.856428,-42.316284,4.942040,어제 뉴스에서 강남 쪽에 좀 많이 걸렸다고 하던데,슬픔,1.643241
8,45.315025,-9.396509,-11.370493,쉴 수가 없어지금 빨리 해오래,슬픔,1.653673
9,47.944767,8.364527,-11.657121,쉬웠는데 긴장하니까 생각이 하나도 안나더라고!,슬픔,1.759773


In [3]:
import plotly.express as px
import plotly.graph_objects as go

# 이상치 데이터프레임 생성 (이전 딕셔너리를 데이터프레임으로 변환)
outliers_df = pd.DataFrame([
    {
        'tsne-3d-1': key[0],
        'tsne-3d-2': key[1],
        'tsne-3d-3': key[2],
        'text': value['text'],
        'emotion': value['emotion'],
        'distance_ratio': value['distance_ratio']
    }
    for key, value in outliers_dict.items()
])

# 클러스터 중심 데이터프레임 생성
# 클러스터 중심 좌표를 미리 추출해서 클러스터 센터 데이터 프레임으로 변환
cluster_centers_df = pd.DataFrame([
    {
        'tsne-3d-1': kmeans.cluster_centers_[0][0],
        'tsne-3d-2': kmeans.cluster_centers_[0][1],
        'tsne-3d-3': kmeans.cluster_centers_[0][2],
        'emotion': emotion
    }
    for emotion in df['emotion'].unique()
])

# 3D 산점도 생성
fig = px.scatter_3d(
    outliers_df, 
    x='tsne-3d-1', 
    y='tsne-3d-2', 
    z='tsne-3d-3',
    color='emotion', 
    hover_data=['text', 'distance_ratio'],
    labels={'color': 'Emotion'},
    title='3D t-SNE 이상치 시각화 및 클러스터 중심'
)

# 클러스터 중심을 무채색으로 추가하고, 레이블을 텍스트로 표시
for _, row in cluster_centers_df.iterrows():
    fig.add_trace(go.Scatter3d(
        x=[row['tsne-3d-1']],
        y=[row['tsne-3d-2']],
        z=[row['tsne-3d-3']],
        mode='markers+text',
        marker=dict(color='gray', size=8, symbol='diamond'),
        text=row['emotion'],  # 클러스터의 감정 레이블을 텍스트로 표시
        textposition='top center',
        name=f"{row['emotion']} 중심"
    ))

# 그래프 출력
fig.show()


In [5]:
import plotly.io as pio

# Plotly의 기본 렌더러 확인
print(pio.renderers.default)

# 필요한 경우 렌더러를 nbformat으로 설정
pio.renderers.default = "browser"

fig.show()

vscode
